In [1]:
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
import string
from nltk.sentiment import SentimentIntensityAnalyzer
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')
article = dataset['train']['article']
summary = dataset['train']['highlights']
ids = dataset['train']['id']

# Define data chunking function
def data_chunks(dataset, chunk_size=40000):
    article = dataset['train']['article']
    summary = dataset['train']['highlights']
    ids = dataset['train']['id']
    chunked_data = []
    for i in range(0, len(article), chunk_size):
        chunk = {
            'id': ids[i:i+chunk_size],
            'article': article[i:i+chunk_size],
            'summary': summary[i:i+chunk_size]
        }
        chunked_data.append(chunk)
        # Save each chunk as a CSV file
        chunk_df = pd.DataFrame(chunk)
        # drop na
        chunk_df = chunk_df.dropna()
        chunk_df.to_csv(f'cnn_chunk_{i//chunk_size}.csv', index=False)
    return chunked_data

chunked_data = data_chunks(dataset)

# Define preprocessing functions
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

def preprocess_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return text

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def remove_punctuation(text):
    exclude = set(string.punctuation)
    return ''.join(char for char in text if char not in exclude)

def clean_fields(df):
    df['article'] = df['article'].apply(lambda x: remove_url(x))
    df['summary'] = df['summary'].apply(lambda x: remove_url(x))

    df['article'] = df['article'].apply(lambda x: remove_html(x))
    df['summary'] = df['summary'].apply(lambda x: remove_html(x))

    df['article'] = df['article'].apply(lambda x: preprocess_text(x))
    df['summary'] = df['summary'].apply(lambda x: preprocess_text(x))

    df['article'] = df['article'].apply(lambda x: remove_punctuation(x))
    df['summary'] = df['summary'].apply(lambda x: remove_punctuation(x))
    return df

# Apply preprocessing to each chunk and save
for i, chunk in enumerate(chunked_data):
    chunk_df = pd.DataFrame(chunk)
    clean_chunk_df = clean_fields(chunk_df)
    clean_chunk_df.to_csv(f'clean_cnn_chunk_{i}.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [9]:
clean_chunk_df = pd.read_csv('clean_cnn_chunk_2.csv')

# Select the first 7000 rows (like with summarization)
clean_chunk_df = clean_chunk_df.head(7000)


In [10]:
import string
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

articles_series = clean_chunk_df['article']
sia = SentimentIntensityAnalyzer()

def get_sentiment(article_text):
    sentiment_scores = sia.polarity_scores(article_text)
    if sentiment_scores['compound'] >= 0.10:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.10:
        return 'negative'
    else:
        return 'neutral'

sentiments = clean_chunk_df['article'].apply(get_sentiment)

# Add sentiment results to DataFrame as a new column
clean_chunk_df['sentiment_vendor'] = sentiments

# Display the DataFrame with sentiment results
print(clean_chunk_df.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                         id  \
0  b1abfc993a55b7b27001e8a692e5ac6f20825d05   
1  006dd02051a6d228fa3841ab555fced21fccdf1e   
2  bc31ad108b7dbc7f8489c4ea2426d3340a912333   
3  a5227658a7f7f58286c9175a8cd471c5e946050a   
4  a9d8a27ef232db714a45bf44c468859ad57422c9   

                                             article  \
0  cnn  flamboyant carnival musician michel marte...   
1  kenyan counterterrorism sources are looking at...   
2  unmanned aerial vehicles arent generally thoug...   
3  washington cnn  one of two photojournalists ki...   
4  cnna nurse who was quarantined against her wil...   

                                             summary sentiment_vendor  
0  new streets fill with jubilant michel martelly...         positive  
1  a norwegian citizen of somali descent is inves...         negative  
2  the uae launched the drones for good award las...         positive  
3  footage of us helicopter attack on two reuters...         negative  
4  nurse kaci hickox

In [11]:
sentiment_counts = clean_chunk_df['sentiment_vendor'].value_counts()

# Display the counts
print("Number of Positive Articles:", sentiment_counts['positive'])
print("Number of Negative Articles:", sentiment_counts['negative'])
print("Number of Neutral Articles:", sentiment_counts['neutral'])

Number of Positive Articles: 3358
Number of Negative Articles: 3570
Number of Neutral Articles: 72


In [12]:
num_examples = 51
clean_chunk_df = pd.read_csv('clean_cnn_chunk_2.csv')[:num_examples]
true_labels = ["positive", "negative", "positive", "negative", "negative", "neutral", "negative", "neutral", "positive", "negative", "neutral", "negative", "neutral", "negative", "negative", "negative", "negative", "negative", "negative", "positive", "positive", "neutral", "neutral", "neutral", "neutral", "positive", "negative", "negative", "neutral", "negative", "neutral", "negative", "negative", "positive", "negative", "negative", "negative", "negative", "positive", "negative", "negative", "positive", "negative", "negative", "negative", "positive", "negative", "positive", "neutral", "negative", "negative"]
# Add a new column 'true_sentiment' with true labels
clean_chunk_df['sentiment_true'] = true_labels
clean_chunk_df['sentiment_vendor'] = sentiments
# Display the DataFrame to verify the new column
print(clean_chunk_df.head())


                                         id  \
0  b1abfc993a55b7b27001e8a692e5ac6f20825d05   
1  006dd02051a6d228fa3841ab555fced21fccdf1e   
2  bc31ad108b7dbc7f8489c4ea2426d3340a912333   
3  a5227658a7f7f58286c9175a8cd471c5e946050a   
4  a9d8a27ef232db714a45bf44c468859ad57422c9   

                                             article  \
0  cnn  flamboyant carnival musician michel marte...   
1  kenyan counterterrorism sources are looking at...   
2  unmanned aerial vehicles arent generally thoug...   
3  washington cnn  one of two photojournalists ki...   
4  cnna nurse who was quarantined against her wil...   

                                             summary sentiment_true  \
0  new streets fill with jubilant michel martelly...       positive   
1  a norwegian citizen of somali descent is inves...       negative   
2  the uae launched the drones for good award las...       positive   
3  footage of us helicopter attack on two reuters...       negative   
4  nurse kaci hickox i do

In [13]:
from sklearn.metrics import accuracy_score, classification_report
y_true = clean_chunk_df['sentiment_true']
y_pred = clean_chunk_df['sentiment_vendor']

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Print classification report for detailed metrics
labels = ['negative', 'neutral', 'positive']
report = classification_report(y_true, y_pred, labels=labels, target_names=labels)
print(report)

Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

    negative       0.83      0.86      0.85        29
     neutral       1.00      0.09      0.17        11
    positive       0.40      0.73      0.52        11

    accuracy                           0.67        51
   macro avg       0.74      0.56      0.51        51
weighted avg       0.78      0.67      0.63        51



In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

true_labels = clean_chunk_df['sentiment_true'].values
# Predicted labels (extracted from the DataFrame)
predicted_labels = clean_chunk_df['sentiment_vendor'].values

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Accuracy: 0.6666666666666666
Precision: 0.7758169934640523
Recall: 0.6666666666666666
F1-Score: 0.6291573308747592


In [ ]:
# summary with cnn bart sentiment analysis

In [21]:
clean_chunk_df = pd.read_csv('best_cnnbart.csv')
# Select the first 7000 rows (like with summarization)
clean_chunk_df = clean_chunk_df.head(7000)

articles_series = clean_chunk_df['generated_summary']
sia = SentimentIntensityAnalyzer()

def get_sentiment(article_text):
    sentiment_scores = sia.polarity_scores(article_text)
    if sentiment_scores['compound'] >= 0.10:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.10:
        return 'negative'
    else:
        return 'neutral'

sentiments = clean_chunk_df['article'].apply(get_sentiment)

# Add sentiment results to DataFrame as a new column
clean_chunk_df['sentiment_vendor'] = sentiments

# Display the DataFrame with sentiment results
print(clean_chunk_df.head())

                                             article  \
0  (CNN) -- Flamboyant carnival musician Michel M...   
1  Kenyan counterterrorism sources are looking at...   
2  Unmanned aerial vehicles aren't generally thou...   
3  Washington (CNN) -- One of two photojournalist...   
4  (CNN)A nurse who was quarantined against her w...   

                                             summary  \
0  NEW: Streets fill with jubilant Michel Martell...   
1  A Norwegian citizen of Somali descent is inves...   
2  The UAE launched the Drones for Good Award las...   
3  Footage of U.S. helicopter attack on two Reute...   
4  Nurse Kaci Hickox: "I don't plan on sticking t...   

                                         id  \
0  b1abfc993a55b7b27001e8a692e5ac6f20825d05   
1  006dd02051a6d228fa3841ab555fced21fccdf1e   
2  bc31ad108b7dbc7f8489c4ea2426d3340a912333   
3  a5227658a7f7f58286c9175a8cd471c5e946050a   
4  a9d8a27ef232db714a45bf44c468859ad57422c9   

                                   generate

In [22]:
sentiment_counts = clean_chunk_df['sentiment_vendor'].value_counts()

# Display the counts
print("Number of Positive Articles:", sentiment_counts['positive'])
print("Number of Negative Articles:", sentiment_counts['negative'])
print("Number of Neutral Articles:", sentiment_counts['neutral'])

Number of Positive Articles: 3372
Number of Negative Articles: 3566
Number of Neutral Articles: 62


In [17]:
num_examples = 49
clean_chunk_df = pd.read_csv('best_cnnbart.csv')[:num_examples]
true_labels = ["positive", "neutral", "positive", "neutral", "negative", "neutral", "negative", "neutral", "positive", "negative", "positive", "negative", "neutral", "neutral",  "neutral",  "negative",  "negative",  "positive",  "negative",  "positive",  "neutral",   "negative",  "neutral",  "neutral",  "negative",  "positive",  "negative",  "neutral",  "neutral",  "negative",  "neutral",  "negative",  "negative",  "positive",  "negative",  "negative",  "neutral",  "negative",  "negative",  "negative",  "negative",  "negative",  "negative",  "negative",  "negative",  "positive",  "negative",  "negative",  "neutral"]
# Add a new column 'true_sentiment' with true labels
clean_chunk_df['sentiment_true'] = true_labels
clean_chunk_df['sentiment_vendor'] = sentiments
# Display the DataFrame to verify the new column
print(clean_chunk_df.head())

                                             article  \
0  (CNN) -- Flamboyant carnival musician Michel M...   
1  Kenyan counterterrorism sources are looking at...   
2  Unmanned aerial vehicles aren't generally thou...   
3  Washington (CNN) -- One of two photojournalist...   
4  (CNN)A nurse who was quarantined against her w...   

                                             summary  \
0  NEW: Streets fill with jubilant Michel Martell...   
1  A Norwegian citizen of Somali descent is inves...   
2  The UAE launched the Drones for Good Award las...   
3  Footage of U.S. helicopter attack on two Reute...   
4  Nurse Kaci Hickox: "I don't plan on sticking t...   

                                         id  \
0  b1abfc993a55b7b27001e8a692e5ac6f20825d05   
1  006dd02051a6d228fa3841ab555fced21fccdf1e   
2  bc31ad108b7dbc7f8489c4ea2426d3340a912333   
3  a5227658a7f7f58286c9175a8cd471c5e946050a   
4  a9d8a27ef232db714a45bf44c468859ad57422c9   

                                   generate

In [18]:
y_true = clean_chunk_df['sentiment_true']
y_pred = clean_chunk_df['sentiment_vendor']

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Print classification report for detailed metrics
labels = ['negative', 'neutral', 'positive']
report = classification_report(y_true, y_pred, labels=labels, target_names=labels)
print(report)

Accuracy: 0.6122448979591837
              precision    recall  f1-score   support

    negative       0.75      0.84      0.79        25
     neutral       1.00      0.07      0.12        15
    positive       0.40      0.89      0.55         9

    accuracy                           0.61        49
   macro avg       0.72      0.60      0.49        49
weighted avg       0.76      0.61      0.54        49



In [19]:
true_labels = clean_chunk_df['sentiment_true'].values
# Predicted labels (extracted from the DataFrame)
predicted_labels = clean_chunk_df['sentiment_vendor'].values

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Accuracy: 0.6122448979591837
Precision: 0.7622448979591837
Recall: 0.6122448979591837
F1-Score: 0.5439150611448222
